# 5 Signs You've Become an Advanced Pythonista Without Even Realizing It
## Time to take credit
![](images/pixabay.jpg)

<figcaption style='text-align: center;'>
Image by <a href="https://pixabay.com/users/summerglow-20203311/?utm_source=link-attribution&amp;utm_medium=referral&amp;utm_campaign=image&amp;utm_content=7120431">Charles Thonney</a> from <a href="https://pixabay.com//?utm_source=link-attribution&amp;utm_medium=referral&amp;utm_campaign=image&amp;utm_content=7120431">Pixabay</a>
</figcaption>